In [ ]:
# %load 8_puzzle_game.py
"""
Artifical Intelligence - MSOE
Author - Gagan Daroach <gagandaroach@gmail.com>
April 26 2019

An implementation of the Puzzle8 Game. 
This file is part of my coursework during my undergrad at Milwaukee School of Engineering.
The purpose of this file is to be used as a illustration and learning tool
    for the hill climbing and a* search techniques with different heuristics.
"""

import numpy as np
import random
import pprint as pp
from enum import Enum, auto


class Direction(Enum):
    RIGHT = auto()
    LEFT = auto()
    DOWN = auto()
    UP = auto()


class Puzzle8:
    """
    Position Locations
    [ [0. 1. 2.]
      [3. 4. 5.]
      [6. 7. 8.] ]
    """
    def __init__(self, pattern = "012345678"):
        self.grid = self.emptyGrid()
        self.load_pattern(pattern)

    def emptyGrid(self):
        return np.zeros((3, 3))

    def load_pattern(self, pattern):
        """pattern format: 012345678"""
        if len(pattern) != 9:
            print("Error - Pattern %s was not the correct length." % pattern)
        else:
            for pos in range(9):
                self.set_tile_at(pos, pattern[pos])

    def dump_pattern(self):
        pattern = ""
        for index in range(9):
            pattern += str(int(self.get_tile_at(index)))
        return pattern

    def __str__(self):
        return ' Grid Printout:\n' + str(self.grid)

    def swap_tiles(self, pos1, pos2):
        """swaps the values of two tiles"""
        temp = self.get_tile_at(pos1)
        self.set_tile_at(pos1, self.get_tile_at(pos2))
        self.set_tile_at(pos2, temp)

    def get_tile_at(self, pos):
        coordinates = self.pos_to_coordinate(pos)
        return int(self.grid[coordinates[1]][coordinates[0]])

    def set_tile_at(self, pos, val):
        coordinates = self.pos_to_coordinate(pos)
        self.grid[coordinates[1]][coordinates[0]] = val

    def pos_to_coordinate(self, pos):
        y = self.pos_row(pos)
        x = self.pos_col(pos)
        return (x, y)

    def pos_row(self, pos):
        return pos // 3

    def pos_col(self, pos):
        return pos % 3

    def find_missing_tile(self):
        return self.find_tile(0)

    def find_tile(self, val):
        for pos in range(9):
            looking_at = self.get_tile_at(pos)
            if looking_at == val:
                return pos
        return None

    def move_missing_tile(self, direction):
        pos = self.find_missing_tile()

        if direction == Direction.UP:
            if (self.pos_row(pos) == 0):
                return False
            else:
                self.swap_tiles(pos, pos - 3)
                return True
        elif direction == Direction.RIGHT:
            if (self.pos_col(pos) == 2):
                return False
            else:
                self.swap_tiles(pos, pos + 1)
                return True
        elif direction == Direction.LEFT:
            if (self.pos_col(pos) == 0):
                return False
            else:
                self.swap_tiles(pos, pos - 1)
                return True
        else:
            if (self.pos_row(pos) == 2):
                return False
            else:
                self.swap_tiles(pos, pos + 3)
                return True

    def randomize_puzzle(self, num_random_moves = 15, debug = False):
        count = 0
        while count < num_random_moves:
            random_direction = random.choice(list(Direction))
            if (self.move_missing_tile(random_direction)):
                if debug: print(self)
                count += 1

    def calculate_misplaced_tiles(self, solution_pattern, debug = False):
        num_correct = 0
        for pos in range(9):
            if int(self.get_tile_at(pos)) == int(solution_pattern[pos]):
                if debug: print(solution_pattern[pos], self.get_tile_at(pos))
                num_correct += 1
        return 9 - num_correct

    def calculate_manhattan_distance(self, solution_pattern, debug = False):
        man_distance = 0
        for pos in range(9):
            my_tile = int(self.get_tile_at(pos))
            sol_tile = int(solution_pattern[pos])
            my_pos = self.find_tile(my_tile)
            sol_pos = self.find_tile(sol_tile)
            distance = self.pos_distance(my_pos, sol_pos)
            man_distance += distance
        return man_distance

    def pos_distance(self, pos1, pos2):
        coord1 = self.pos_to_coordinate(pos1)
        coord2 = self.pos_to_coordinate(pos2)
        deltaX = abs(coord1[0] - coord2[0])
        deltaY = abs(coord1[1] - coord2[1])
        return deltaX + deltaY

# end of class, start of my tests
# used during development to see if stuff was working right

def calculate_manhattan_distance_test():
    puzzle = Puzzle8()
    print('should be 0: ', puzzle.calculate_manhattan_distance("012345678", True))
    print('should be something: ', puzzle.calculate_manhattan_distance("876543210", True))
    print('should be something smaller; ', puzzle.calculate_manhattan_distance("012348765", True))

def calculate_misplaced_tiles_test():
    puzzle = Puzzle8()
    print('should be 0: ', puzzle.calculate_misplaced_tiles("012345678", True))
    print('should be 8: ', puzzle.calculate_misplaced_tiles("876543210", True))
    print('should be half; ', puzzle.calculate_misplaced_tiles("012348765", True))


def dump_pattern_test():
    puzzle = Puzzle8()
    print(puzzle.dump_pattern(), True)

def move_test():
    puzzle = Puzzle8()
    print(puzzle.move_missing_tile(Direction.RIGHT))
    print(puzzle)
    print(puzzle.move_missing_tile(Direction.RIGHT))
    print(puzzle.move_missing_tile(Direction.RIGHT))
    print(puzzle.move_missing_tile(Direction.DOWN))
    print(puzzle)

def swap_test():
    puzzle = Puzzle8()
    puzzle.swap_tiles(0, 8)
    puzzle.swap_tiles(0, 3)
    puzzle.swap_tiles(8, 4)
    print(puzzle)
    print(puzzle.find_missing_tile())
    print(puzzle.find_missing_tile() == 4)

def load_pattern_test():
    puzzle = Puzzle8()
    pattern = "158764302"
    puzzle.load_pattern(pattern)
    print(pattern, puzzle)

def randomize_test():
    puzzle = Puzzle8()
    puzzle.randomize_puzzle()
    print(puzzle)

if __name__ == "__main__":
    calculate_manhattan_distance_test()

In [2]:
puzzle = Puzzle8()
puzzle.randomize_puzzle()
puzzle.dump_pattern()

'632715408'

In [3]:
pattern = _
print(pattern)

632715408


```
Discrete Space Hill Climbing Algorithm
   currentNode = startNode;
   loop do
      L = NEIGHBORS(currentNode);
      nextEval = -INF;
      nextNode = NULL;
      for all x in L 
         if (EVAL(x) > nextEval)
              nextNode = x;
              nextEval = EVAL(x);
      if nextEval <= EVAL(currentNode)
         //Return current node since no better neighbors exist
         return currentNode;
      currentNode = nextNode;
```

In [4]:
def calculate_possible_heuristics(current_pattern, solution_pattern):
    heuristics = {}
    for enum in Direction:
        duplicate = Puzzle8(current_pattern)
        if duplicate.move_missing_tile(enum):
            heuristic_value = duplicate.calculate_misplaced_tiles(solution_pattern)
        else:
            heuristic_value = 4200 #arbitrarily set to high value bc illegal move.
        heuristics[enum] = heuristic_value
    return heuristics

In [5]:
calculate_possible_heuristics("012345678", "012345678")

{<Direction.RIGHT: 1>: 2,
 <Direction.LEFT: 2>: 4200,
 <Direction.DOWN: 3>: 2,
 <Direction.UP: 4>: 4200}

In [6]:
def determine_best_possible_heuristic(heuristics):
    lowest_enum_value = 100000
    best_heuristic = None
    for enum in Direction:
        if heuristics[enum] < lowest_enum_value:
            best_heuristic = enum
            lowest_enum_value = heuristics[enum]
    return best_heuristic

In [7]:
poss_heuristics = calculate_possible_heuristics("012345678", "012345678")
determine_best_possible_heuristic(poss_heuristics)

<Direction.RIGHT: 1>

In [8]:
def climb_that_hill(init_pattern, sol_pattern = "012345678", max_moves = 100):
    print("Hill Climb Search:\n\tinit pattern: %s\n\ttarget pattern: %s" % (init_pattern, sol_pattern))
    heuristic = 1
    num_moves = 0
    current_pattern = init_pattern
    while heuristic != 0 and num_moves < max_moves:
        poss_heuristics = calculate_possible_heuristics(current_pattern, sol_pattern)
        enum_best_move = determine_best_possible_heuristic(poss_heuristics)
        puzzle = Puzzle8(current_pattern)
        puzzle.move_missing_tile(enum_best_move)
        heuristic = puzzle.calculate_misplaced_tiles(sol_pattern)
        current_pattern = puzzle.dump_pattern()
        num_moves += 1
    return num_moves
        

In [9]:
climb_that_hill(pattern)

Hill Climb Search:
	init pattern: 632715408
	target pattern: 012345678


100

In [10]:
puzzle2 = Puzzle8()

In [11]:
puzzle2.randomize_puzzle(150)

In [12]:
puzzle2.dump_pattern()

'563742018'

In [13]:
pattern_150_random_moves = _

In [14]:
pattern_150_random_moves

'563742018'

In [15]:
climb_that_hill(pattern_150_random_moves)

Hill Climb Search:
	init pattern: 563742018
	target pattern: 012345678


100

In [16]:
climb_that_hill(pattern_150_random_moves, max_moves=250)

Hill Climb Search:
	init pattern: 563742018
	target pattern: 012345678


250

## obesrvations -> random hill climbing misplaced tile heuristic

* with 15 random moves done to a puzzle, the search strategy was able to find a solution in 5 moves. Looking at the pattern, the tile did not move far from the top left, and the last row of the puzzle was unchanged.

* with 150 random moves done to the puzzle, the search strategy was not able to find a solution in 100 moves. I tried upping the number of moves to 250, but the results did not change. My guess is the hill climbing algorithm may have reached a recursive loop.

In [17]:
def calculate_possible_heuristics_manhattan(current_pattern, solution_pattern):
    heuristics = {}
    for enum in Direction:
        duplicate = Puzzle8(current_pattern)
        if duplicate.move_missing_tile(enum):
            heuristic_value = duplicate.calculate_manhattan_distance(solution_pattern)
        else:
            heuristic_value = 4200 #arbitrarily set to high value bc illegal move.
        heuristics[enum] = heuristic_value
    return heuristics

In [18]:
calculate_possible_heuristics_manhattan("123456780","123456780")

{<Direction.RIGHT: 1>: 4200,
 <Direction.LEFT: 2>: 2,
 <Direction.DOWN: 3>: 4200,
 <Direction.UP: 4>: 2}

In [19]:
calculate_possible_heuristics_manhattan("087654321","123456780")

{<Direction.RIGHT: 1>: 24,
 <Direction.LEFT: 2>: 4200,
 <Direction.DOWN: 3>: 24,
 <Direction.UP: 4>: 4200}

In [20]:
determine_best_possible_heuristic(calculate_possible_heuristics_manhattan("123456780","012345678"))

<Direction.LEFT: 2>

In [21]:
determine_best_possible_heuristic(calculate_possible_heuristics_manhattan("012345678","012345678"))

<Direction.RIGHT: 1>

In [22]:
determine_best_possible_heuristic(calculate_possible_heuristics_manhattan("210345678","012345678"))

<Direction.LEFT: 2>

In [23]:
def climb_that_hill_manhattan(init_pattern, sol_pattern = "012345678", max_moves = 100):
    print("Hill Climb Search:\n\tinit pattern: %s\n\ttarget pattern: %s" % (init_pattern, sol_pattern))
    heuristic = 1
    num_moves = 0
    current_pattern = init_pattern
    while heuristic != 0 and num_moves < max_moves:
        if current_pattern == sol_pattern:
            return num_moves
        poss_heuristics = calculate_possible_heuristics_manhattan(current_pattern, sol_pattern)
        enum_best_move = determine_best_possible_heuristic(poss_heuristics)
        puzzle = Puzzle8(current_pattern)
        puzzle.move_missing_tile(enum_best_move)
        heuristic = puzzle.calculate_manhattan_distance(sol_pattern)
        current_pattern = puzzle.dump_pattern()
        num_moves += 1
    return num_moves

In [24]:
climb_that_hill_manhattan(pattern)

Hill Climb Search:
	init pattern: 632715408
	target pattern: 012345678


100

In [25]:
climb_that_hill_manhattan("012345678")

Hill Climb Search:
	init pattern: 012345678
	target pattern: 012345678


0

In [26]:
climb_that_hill_manhattan(pattern_150_random_moves, max_moves=250)

Hill Climb Search:
	init pattern: 563742018
	target pattern: 012345678


250

In [27]:
climb_that_hill_manhattan(pattern_150_random_moves)

Hill Climb Search:
	init pattern: 563742018
	target pattern: 012345678


100

In [28]:
puzzle3 = Puzzle8()
puzzle3.randomize_puzzle(15)
climb_that_hill_manhattan(pattern_150_random_moves)

Hill Climb Search:
	init pattern: 563742018
	target pattern: 012345678


100